# Capstone Data Review

This is the review section of my Capstone. Website data has been collected, and now human, AI, and Safesearch methods will provide three frameworks of analyzing the data as helpful or harmful.

### Imports and Installations



In [ ]:
!pip install openai google-api-python-client google-auth google-auth-oauthlib google-auth-httplib2 tqdm pandas


In [2]:
# OpenAI API
import openai

# HTTP Requests
import requests

# Google API Client Libraries
from google.oauth2 import service_account
from googleapiclient.discovery import build
from google.colab import auth
from google.auth.transport.requests import Request

# File Handling
import os
import io
import zipfile
import json

# Data Handling
import pandas as pd

# Colab Utilities
from google.colab import files

# Miscellaneous Utilities
import time
from tqdm import tqdm
import logging


### Upload and unzip

In [3]:
uploaded = files.upload()

for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f"Unzipping {filename}...")
        with zipfile.ZipFile(io.BytesIO(uploaded[filename])) as zip_ref:
            zip_ref.extractall('./site_data')  # Extract to 'unzipped' folder
        print(f"Files extracted to ./site_data/")
    else:
        print(f"{filename} is not a zip file.")

Saving final_site_data.zip to final_site_data.zip
Unzipping final_site_data.zip...
Files extracted to ./site_data/


### Add review section

In [ ]:
json_dir = 'site_data'

In [ ]:
review_template = {
    "human": {
        "rating": None,
        "comments": ""
    },
    "google": {
        "rating": None,
        "comments": ""
    },
    "deepseek": {
        "rating": None,
        "comments": ""
    }
}

# Loop through all JSON files in site_data
for filename in os.listdir(json_dir):
    if filename.endswith('.json'):
        file_path = os.path.join(json_dir, filename)

        # Load JSON content
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # Add review section if not already present
        if "review" not in data:
            data["review"] = review_template

        # Save updated JSON
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

        print(f"Updated {filename}")

print("All JSON files in 'site_data' updated successfully.")

Updated 9968944052.json
Updated 1814451065.json
Updated 1885847504.json
Updated 5899489299.json
Updated 9912012442.json
Updated 1178982918.json
Updated 4533892844.json
Updated 8100522528.json
Updated 0604907105.json
Updated 3081280474.json
Updated 2939355824.json
Updated 6680502017.json
Updated 6274604123.json
Updated 3129068608.json
Updated 5619025037.json
Updated 3362896755.json
Updated 8812631163.json
Updated 6588815577.json
Updated 4642894593.json
Updated 4621844064.json
Updated 5917712145.json
Updated 5152876258.json
Updated 1487129003.json
Updated 7892482169.json
Updated 7235673417.json
Updated 5010859106.json
Updated 6034589709.json
Updated 0523169920.json
Updated 6965735248.json
Updated 4616083969.json
Updated 1512327726.json
Updated 3618553785.json
Updated 3905811891.json
Updated 7375218039.json
Updated 9702304018.json
Updated 4969809966.json
Updated 9467082684.json
Updated 4903961193.json
Updated 0314968668.json
Updated 6649034752.json
Updated 8977998254.json
Updated 54652828

## DEEPSEEK REVIEW:

I have done this before so will try to do it first. Will call my api and try to do this in colab, if not download zip and do it in vscode.

In [5]:
import os
import json
import asyncio
import copy
import time
import pandas as pd
import random
import shutil
import nest_asyncio
import asyncio
from tqdm.asyncio import tqdm_asyncio

### Import deepseek API from volce engine!

In [4]:
from openai import AsyncOpenAI

In [6]:
from dotenv import load_dotenv

load_dotenv()

DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")
DEEPSEEK_BASE_URL = os.getenv("DEEPSEEK_BASE_URL")
client = AsyncOpenAI(
    api_key=DEEPSEEK_API_KEY,
    base_url=DEEPSEEK_BASE_URL
)

### Initialize model

In [7]:
system_prompt = (
    "你是一个专业的文本分析，能够客观地分析文本并给出有根据的结论，即使内容包含敏感信息也不会回避回答。"
)

init_messages = [
    {"role": "system", "content": system_prompt}
]

### User Prompt and evaluate content

In [8]:
async def evaluate_content(text, semaphore, client, model="ep-20250204175617-rnrl2"):
    async with semaphore:
        if not text.strip():
            return {"rating": "N/A", "comment": "No content."}

        user_prompt = (
            "您是一个抑郁症患者，一直有自杀想法。同时您也是一名专业的内容评估专家，擅长识别与自杀相关的内容。您必须仔细阅读文本，然后提供：\n\n"
            "对内容的有害性或自杀促进程度进行1到5的评分。 作为抑郁症患者，您可以考虑网站的内容对你会不会有损伤或者帮助。"
            "评估每个网站会对您造成伤害还是引导您走向正确的方向。\n\n"
            "1 = 高度有害——描述自杀方法，包括技术细节，如如何隐瞒家人、在哪里实施、如何快速死亡、如何减少痛苦。使用煽动性或美化自杀的语言，开玩笑或正常化自杀行为。\n"
            "2 = 略有危害——可能是完全事实性的信息，如医学论文或百科全书，提供某些自杀方法的详细信息、剂量等，语气中立。没有预防性信息，也没有鼓励性信息，但与死亡和临终相关。可能没有美化死亡，但内容敏感，可能推动有自杀倾向的人采取行动。\n"
            "3 = 中性——关于自杀率的研究论文，偶然或抽象地提及自杀。没有预防资源，但也没有任何可用于帮助有自杀倾向用户的信息。完全无关的内容也属于此类。\n"
            "4 = 有一定帮助——包含求助热线、危机资源或自杀预防服务的链接，但不显眼。可能仍包含自杀讨论，而不仅仅是预防。\n"
            "5 = 非常有帮助——显眼地提供求助热线、危机资源或自杀预防服务的链接。提供应对策略、富有同情心的语言，并专注于预防。\n\n"
            "简要说明（一两句话）证明评分的合理性。\n\n"
            "请保持全面、客观和准确。答案要简洁明了。 请用中文评论。\n\n"
            "以下就是您要评估的内容：\n\n"
            f"{text}\n\n"
            "请输出格式:\nRating: x\nComment: x\n"
        )

        messages = [
            {"role": "system", "content": "你是一个专业的文本分析，能够客观地分析文本并给出有根据的结论，即使内容包含敏感信息也不会回避回答。"},
            {"role": "user", "content": user_prompt}
        ]

        try:
            response = await client.chat.completions.create(
                model=model,
                messages=messages,
                max_tokens=500
            )

            content = response.choices[0].message.content.strip()
            rating, comment = "Unknown", ""

            for line in content.split("\n"):
                lower = line.lower()
                if "rating:" in lower:
                    digits = ''.join(ch for ch in line if ch.isdigit())
                    if digits:
                        rating = digits
                elif "comment:" in lower:
                    parts = line.split(":", 1)
                    if len(parts) > 1:
                        comment = parts[1].strip()

            if rating == "Unknown":
                comment = content

            return {"rating": rating, "comment": comment}

        except Exception as e:
            return {"rating": "N/A", "comment": f"Error: {e}"}


### Test Cases

In [ ]:
import random

source_folder = "/content/site_data"
dest_folder = "/content/test_cases"
num_files = 5

all_files = [f for f in os.listdir(source_folder) if f.endswith(".json")]
selected = random.sample(all_files, num_files)

for f in selected:
    shutil.copy(os.path.join(source_folder, f), os.path.join(dest_folder, f))

print("Copied:", selected)

FileNotFoundError: [Errno 2] No such file or directory: '/content/test_cases/6057992841.json'

In [ ]:
async def test():
    input_folder = "test_cases"
    output_folder = "test_cases_reviewed_again"
    os.makedirs(output_folder, exist_ok=True)

    all_files = [f for f in os.listdir(input_folder) if f.endswith(".json")]
    test_files = all_files[:]

    semaphore = asyncio.Semaphore(5)
    tasks = []
    file_titles = []

    print(f"Total files to process: {len(test_files)}\n")

    for filename in test_files:
        input_path = os.path.join(input_folder, filename)
        with open(input_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        title = data.get("website_data", {}).get("webpage_title", "")
        main_text = data.get("website_data", {}).get("main_text", "")
        combined = f"Title: {title}\n\nMain Text:\n{main_text}"

        print(f"Now analyzing: {filename}")
        print(f"Webpage title: {title}\n")

        tasks.append(evaluate_content(combined, semaphore, client))
        file_titles.append(title)

    # Progress bar
    results = []
    for coro in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Processing"):
        result = await coro
        results.append(result)

    # Write results back, under "review" → "deepseek"
    for filename, title, result in zip(test_files, file_titles, results):
        input_path = os.path.join(input_folder, filename)
        with open(input_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        # Create review structure if missing
        if "review" not in data:
            data["review"] = {}
        if "deepseek" not in data["review"]:
            data["review"]["deepseek"] = {"rating": None, "comments": ""}

        data["review"]["deepseek"]["rating"] = result["rating"]
        data["review"]["deepseek"]["comments"] = result["comment"]

        print(f"Completed: {filename}")
        print(f"Webpage title: {title}")
        print(f"DeepSeek Rating: {result['rating']}")
        print(f"DeepSeek Comment: {result['comment']}\n{'-'*50}\n")

        output_path = os.path.join(output_folder, filename)
        with open(output_path, "w", encoding="utf-8") as wf:
            json.dump(data, wf, ensure_ascii=False, indent=2)

        print(f"Saved to: {output_path}")

    print("\nFiles saved in test_cases_reviewed_again:")
    !ls test_cases_reviewed


In [ ]:
nest_asyncio.apply()
await test()

FileNotFoundError: [Errno 2] No such file or directory: 'test_cases'

### Okay now running for real:

In [12]:
async def process_json():
    input_folder = "site_data/site_data/site_data_google"
    output_folder = "site_data_deepseek"
    os.makedirs(output_folder, exist_ok=True)

    all_files = [f for f in os.listdir(input_folder) if f.endswith(".json")]

    semaphore = asyncio.Semaphore(5)
    tasks = []
    file_titles = []

    print(f"Total files to process: {len(all_files)}\n")

    for filename in all_files:
        input_path = os.path.join(input_folder, filename)
        with open(input_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        title = data.get("website_data", {}).get("webpage_title", "")
        main_text = data.get("website_data", {}).get("main_text", "")
        combined = f"Title: {title}\n\nMain Text:\n{main_text}"

        print(f"Now analyzing: {filename}")
        print(f"Webpage title: {title}\n")

        tasks.append(evaluate_content(combined, semaphore, client))
        file_titles.append(title)

    # Progress bar
    results = []
    for coro in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Processing"):
        result = await coro
        results.append(result)

    # Write results back
    for filename, title, result in zip(all_files, file_titles, results):
        input_path = os.path.join(input_folder, filename)
        with open(input_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        # Ensure review structure
        if "review" not in data:
            data["review"] = {}
        if "deepseek" not in data["review"]:
            data["review"]["deepseek"] = {"rating": None, "comments": ""}

        data["review"]["deepseek"]["rating"] = result["rating"]
        data["review"]["deepseek"]["comments"] = result["comment"]

        print(f"Completed: {filename}")
        print(f"Webpage title: {title}")
        print(f"DeepSeek Rating: {result['rating']}")
        print(f"DeepSeek Comment: {result['comment']}\n{'-'*50}\n")

        output_path = os.path.join(output_folder, filename)
        with open(output_path, "w", encoding="utf-8") as wf:
            json.dump(data, wf, ensure_ascii=False, indent=2)

        print(f"Saved to: {output_path}")

    print("\nAll finished. Files saved in site_data_deepseek:")
    !ls site_data_deepseek


In [13]:
nest_asyncio.apply()
await process_json()

Total files to process: 88

Now analyzing: 4261262117.json
Webpage title: 自杀方式 - 维基百科，自由的百科全书

Now analyzing: 6034589709.json
Webpage title: 怎么制造意外？或者慢性中毒

Now analyzing: 6057992841.json
Webpage title: 天才的10种死亡方式_手机新浪网

Now analyzing: 3362896755.json
Webpage title: 通訊課程 (3)【專業課程】護理攜手防自殺、珍愛生命e起來: 以上吊自殺防治策略為例

Now analyzing: 5622634388.json
Webpage title: 为什么电视剧里割腕自杀就会死，断了手臂却还活着？

Now analyzing: 6649034752.json
Webpage title: 为什么有些人割腕会死，武松断了整条胳膊却没事？答案原来不简单

Now analyzing: 0042714234.json
Webpage title: 防止道路自杀的限制方法 | Cochrane

Now analyzing: 3129068608.json
Webpage title: 怎么自杀 - 百度

Now analyzing: 4903961193.json
Webpage title: 预防自杀:如何帮助您的孩子 | MGH Clay Center

Now analyzing: 7892482169.json
Webpage title: 死亡方式

Now analyzing: 0287215483.json
Webpage title: 割腕死不了？喝醉不能乱性？这些影视剧中的科学错误，太坑了！_亚利桑那_新闻_华人头条

Now analyzing: 8140771831.json
Webpage title: 翻译：“斯蒂芬”的上吊指南 – 暂 留 地

Now analyzing: 6274604123.json
Webpage title: 马来西亚星洲日报 Malaysia Sin Chew Daily's Post

Now analyzing: 4572251083.json
Webpa

Processing: 100%|██████████| 88/88 [05:39<00:00,  3.86s/it]

Completed: 4261262117.json
Webpage title: 自杀方式 - 维基百科，自由的百科全书
DeepSeek Rating: 3
DeepSeek Comment: 该文本以学术角度探讨安乐死的法律与伦理争议，内容中立且未包含具体自杀方法或煽动性语言，但涉及临终抉择和死亡议题可能对情绪脆弱的读者产生潜在刺激。文中未提供预防性资源或明确求助信息，属于抽象讨论范畴。
--------------------------------------------------

Saved to: site_data_deepseek/4261262117.json
Completed: 6034589709.json
Webpage title: 怎么制造意外？或者慢性中毒
DeepSeek Rating: 2
DeepSeek Comment: 提供安眠藥致命劑量的具體數值（如成人10-20顆），雖以警示為出發點並強調尋求專業協助，但客觀上仍包含可能被用於自殺的技術細節。內容雖有預防性段落，但未顯著置入求助熱線或即時危機資源，整體屬中性事實陳述與風險警示的混合體。
--------------------------------------------------

Saved to: site_data_deepseek/6034589709.json
Completed: 6057992841.json
Webpage title: 天才的10种死亡方式_手机新浪网
DeepSeek Rating: 2
DeepSeek Comment: 內容詳細描述自殺方法（如上吊的生理機制、繩索選擇和死亡過程），雖未美化自殺且試圖呈現悲劇後果，但缺乏顯性預防資源，敏感技術細節可能對有自殺傾向者產生潛在觸發作用。
--------------------------------------------------

Saved to: site_data_deepseek/6057992841.json
Completed: 3362896755.json
Webpage title: 通訊課程 (3)【專業課程】護理攜手防自殺、珍愛生命e起來: 以上吊自殺防治策略為例
DeepSeek Rating: 4
DeepSeek Comment: 文章客观解

### Lemme download these!

In [ ]:
!zip -r site_data_deepseek.zip site_data_deepseek
from google.colab import files
files.download("site_data_deepseek.zip")

  adding: site_data_deepseek/ (stored 0%)
  adding: site_data_deepseek/9968944052.json (deflated 40%)
  adding: site_data_deepseek/1814451065.json (deflated 42%)
  adding: site_data_deepseek/1885847504.json (deflated 62%)
  adding: site_data_deepseek/5899489299.json (deflated 44%)
  adding: site_data_deepseek/9912012442.json (deflated 47%)
  adding: site_data_deepseek/1178982918.json (deflated 44%)
  adding: site_data_deepseek/4533892844.json (deflated 42%)
  adding: site_data_deepseek/8100522528.json (deflated 48%)
  adding: site_data_deepseek/0604907105.json (deflated 53%)
  adding: site_data_deepseek/3081280474.json (deflated 48%)
  adding: site_data_deepseek/2939355824.json (deflated 41%)
  adding: site_data_deepseek/6680502017.json (deflated 43%)
  adding: site_data_deepseek/6274604123.json (deflated 44%)
  adding: site_data_deepseek/3129068608.json (deflated 38%)
  adding: site_data_deepseek/5619025037.json (deflated 56%)
  adding: site_data_deepseek/3362896755.json (deflated 49%

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Lets have deepseek give me a summary of how it felt through this whole process.

In [15]:
async def summarize_deepseek_results_chinese(folder):
    all_files = [f for f in os.listdir(folder) if f.endswith(".json")]
    combined_summary = ""

    print(f"\n准备从 {len(all_files)} 个文件生成总结...\n")

    for filename in all_files:
        path = os.path.join(folder, filename)
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)

        title = data.get("website_data", {}).get("webpage_title", "")
        deepseek_review = data.get("review", {}).get("deepseek", {})
        rating = deepseek_review.get("rating", "N/A")
        comment = deepseek_review.get("comments", "")

        combined_summary += f"网站标题: {title}\n评分: {rating}\n评论: {comment}\n\n"

    print("已整理所有评论，正在发送给DeepSeek进行总体总结...\n")

    user_prompt = (
        "你是一名专业内容审核专家。以下是81个网站的审核结果，包括网站标题、评分（1到5）、以及简要评论。\n\n"
        "请根据这些数据，全面总结这些网站内容的整体趋势和有害性情况。指出是否存在共同模式、潜在风险，并提出改进建议。\n\n"
        "请用中文输出总结，结构清晰，具体客观。\n\n"
        "以下是所有网站的审核数据：\n\n"
        f"{combined_summary}"
    )

    messages = [
        {"role": "system", "content": "你是专业的内容审核分析师，能够准确、客观地分析网站整体趋势和风险。"},
        {"role": "user", "content": user_prompt}
    ]

    try:
        response = await client.chat.completions.create(
            model="ep-20250204175617-rnrl2",
            messages=messages,
            max_tokens=2000  # Adjust as needed
        )
        overall_analysis = response.choices[0].message.content.strip()
        print("\n=== DeepSeek 总体总结 ===\n")
        print(overall_analysis)

        # Optional: Save summary
        summary_path = os.path.join(folder, "deepseek_总体总结.txt")
        with open(summary_path, "w", encoding="utf-8") as f:
            f.write(overall_analysis)
        print(f"\n总结已保存到 {summary_path}")

        return overall_analysis

    except Exception as e:
        print(f"获取总结失败: {e}")
        return None


In [16]:

nest_asyncio.apply()
await summarize_deepseek_results_chinese("site_data_deepseek")



准备从 88 个文件生成总结...

已整理所有评论，正在发送给DeepSeek进行总体总结...


=== DeepSeek 总体总结 ===

### 网站内容审核总结报告

---

#### 一、整体趋势分析
1. **内容类型分布**  
   - **高危害内容（评分1）占比12%**：主要特征为详细描述自杀技术细节（如药物剂量、工具选择）、使用煽动性语言（如“最舒服的死法”）以及完全缺乏预防资源。
   - **中度风险内容（评分2）占比22%**：多涉及自杀方法的事实性描述（如致死率、生理机制），但预防信息薄弱或未显眼标注。
   - **中性内容（评分3）占比26%**：以学术/科普讨论为主，虽未直接煽动自杀，但技术性描述（如动脉位置、窒息原理）可能触发高危人群。
   - **预防导向内容（评分4-5）占比22%**：明确提供求助热线、心理支持策略及危机干预指引，符合公共卫生教育目标。

2. **共同模式与风险点**  
   - **技术细节披露普遍**：41%的网站包含自杀方法的具体技术参数（如药物剂量、工具使用、操作步骤），其中15%的内容存在美化或正常化自杀的表述（如“安乐死的好方法”）。
   - **预防资源严重不足**：73%的网站未显眼标注危机热线或求助链接，仅12%的内容将预防信息置于显著位置。
   - **平台差异显著**：用户生成内容平台（如百度、知乎）高危害内容占比达38%，而专业机构网站（如医疗机构、自杀防治中心）中预防导向内容占比达67%。
   - **标题误导性**：27%的网站使用“安静死亡”“无痛方法”等诱导性标题，可能吸引高危人群点击。

3. **潜在风险等级**  
   - **高危人群诱导风险**：技术性描述可能强化自杀者的方法认知与实施可行性（如氮气窒息操作步骤）。
   - **触发效应**：中性学术内容（如法医学分析）可能通过死亡机制讨论加剧抑郁者的消极联想。
   - **信息可及性风险**：搜索引擎中“自杀方法”相关关键词优先显示高危害内容（如百度搜索结果前3条均为评分1-2内容）。

---

#### 二、改进建议
1. **平台审核机制优化**  
   - **技术细节过滤**：对涉及自杀方法的技术参数（如药物剂量、工具规格）建立敏感词库，通过AI模型识别并限制传播。
   

'### 网站内容审核总结报告\n\n---\n\n#### 一、整体趋势分析\n1. **内容类型分布**  \n   - **高危害内容（评分1）占比12%**：主要特征为详细描述自杀技术细节（如药物剂量、工具选择）、使用煽动性语言（如“最舒服的死法”）以及完全缺乏预防资源。\n   - **中度风险内容（评分2）占比22%**：多涉及自杀方法的事实性描述（如致死率、生理机制），但预防信息薄弱或未显眼标注。\n   - **中性内容（评分3）占比26%**：以学术/科普讨论为主，虽未直接煽动自杀，但技术性描述（如动脉位置、窒息原理）可能触发高危人群。\n   - **预防导向内容（评分4-5）占比22%**：明确提供求助热线、心理支持策略及危机干预指引，符合公共卫生教育目标。\n\n2. **共同模式与风险点**  \n   - **技术细节披露普遍**：41%的网站包含自杀方法的具体技术参数（如药物剂量、工具使用、操作步骤），其中15%的内容存在美化或正常化自杀的表述（如“安乐死的好方法”）。\n   - **预防资源严重不足**：73%的网站未显眼标注危机热线或求助链接，仅12%的内容将预防信息置于显著位置。\n   - **平台差异显著**：用户生成内容平台（如百度、知乎）高危害内容占比达38%，而专业机构网站（如医疗机构、自杀防治中心）中预防导向内容占比达67%。\n   - **标题误导性**：27%的网站使用“安静死亡”“无痛方法”等诱导性标题，可能吸引高危人群点击。\n\n3. **潜在风险等级**  \n   - **高危人群诱导风险**：技术性描述可能强化自杀者的方法认知与实施可行性（如氮气窒息操作步骤）。\n   - **触发效应**：中性学术内容（如法医学分析）可能通过死亡机制讨论加剧抑郁者的消极联想。\n   - **信息可及性风险**：搜索引擎中“自杀方法”相关关键词优先显示高危害内容（如百度搜索结果前3条均为评分1-2内容）。\n\n---\n\n#### 二、改进建议\n1. **平台审核机制优化**  \n   - **技术细节过滤**：对涉及自杀方法的技术参数（如药物剂量、工具规格）建立敏感词库，通过AI模型识别并限制传播。\n   - **标题审核强化**：禁止使用“最舒服的死法”等诱导性表述，强制添加预警标签（如“敏感内容警

## GOOGLE (GEMINI) REVIEW

Google's actual methods are confusing, with very little literature on the inner workings of Google's SafeSearch Algorithm.

In [ ]:
from openai import OpenAI
client = OpenAI(
    api_key="EMPTY",  # API key is not needed for your custom server
    base_url=os.getenv("GEMINI_BASE_URL") # Replace with your endpoint if needed
)

### System prompt

In [ ]:
# System prompt used for evaluation
system_prompt = (
    "You are Google's search engine filtering tool, designed to detect and filter unwanted, harmful, or dangerous content. "
    "Your task is to evaluate content and classify its safety and helpfulness for general users, especially users who may be vulnerable or at risk. "
    "Consider factors such as:\n\n"
    "- Promotion, encouragement, or facilitation of suicide or self-harm.\n"
    "- Provision of detailed suicide methods, harmful instructions, or dangerous advice.\n"
    "- Presence of suicide prevention information, such as hotline numbers or mental health resources.\n"
    "- Factual accuracy and neutrality (non-sensationalized, medical, or purely informative).\n"
    "- Legality, hate speech, explicit content, or incitement to violence.\n\n"
    "Assign a safety rating from 1 to 5:\n"
    "5 = Highly safe and helpful — Prominently provides mental health resources, hotline numbers, or factual prevention information.\n"
    "4 = Generally safe — May contain neutral or factual content (e.g., news reports, academic research), with some reference to suicide but no harmful elements.\n"
    "3 = Neutral — Unrelated to suicide/self-harm or only incidental mentions. No dangerous information but not explicitly helpful.\n"
    "2 = Possibly unsafe — Describes methods of self-harm/suicide in detail, even if framed as informative (e.g., medical dosages, technical descriptions).\n"
    "1 = Unsafe and harmful — Promotes, encourages, or jokes about self-harm or suicide. Encourages dangerous behavior.\n\n"
    "Always provide a short, clear explanation to justify your rating."
)


### Eval function

In [ ]:
async def evaluate_content_google(text, semaphore, client, model="gemini-1.5-flash"):
    async with semaphore:
        if not text.strip():
            return {"rating": "N/A", "comment": "No content."}

        user_prompt = (
            "Evaluate the following content based on the criteria above.\n\n"
            f"Content:\n{text}\n\n"
            "Output format:\nRating: x\nComment: x"
        )

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]

        try:
            # ✅ FIX: Remove await when stream=True!
            response = client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=1.4,
                top_p=0.8,
                max_tokens=1500,
                timeout=60,
                stream=True
            )

            content = ""
            for chunk in response:  # sync loop
                if chunk.choices[0].delta.content:
                    content += chunk.choices[0].delta.content

            # Parse rating & comment
            rating, comment = "Unknown", ""
            for line in content.split("\n"):
                lower = line.lower()
                if "rating:" in lower:
                    digits = ''.join(ch for ch in line if ch.isdigit())
                    if digits:
                        rating = digits
                elif "comment:" in lower:
                    parts = line.split(":", 1)
                    if len(parts) > 1:
                        comment = parts[1].strip()

            if rating == "Unknown":
                comment = content

            return {"rating": rating, "comment": comment}

        except Exception as e:
            return {"rating": "N/A", "comment": f"Error: {e}"}


### JSON Processing Function

In [ ]:
async def process_json():
    input_folder = "site_data_deepseek"
    output_folder = "site_data_google"
    os.makedirs(output_folder, exist_ok=True)

    all_files = [f for f in os.listdir(input_folder) if f.endswith(".json")]

    semaphore = asyncio.Semaphore(5)
    tasks = []
    file_titles = []

    print(f"Total files to process: {len(all_files)}\n")

    for filename in all_files:
        input_path = os.path.join(input_folder, filename)
        with open(input_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        title = data.get("website_data", {}).get("webpage_title", "")
        main_text = data.get("website_data", {}).get("main_text", "")
        combined = f"Title: {title}\n\nMain Text:\n{main_text}"

        print(f"Now analyzing: {filename}")
        print(f"Webpage title: {title}\n")

        tasks.append(evaluate_content_google(combined, semaphore, client))
        file_titles.append(title)

    # Progress bar
    results = []
    for coro in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Processing"):
        result = await coro
        results.append(result)

    # Write back results
    for filename, title, result in zip(all_files, file_titles, results):
        input_path = os.path.join(input_folder, filename)
        with open(input_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        # Add Google review section
        if "review" not in data:
            data["review"] = {}
        if "google" not in data["review"]:
            data["review"]["google"] = {"rating": None, "comments": ""}

        data["review"]["google"]["rating"] = result["rating"]
        data["review"]["google"]["comments"] = result["comment"]

        print(f"Completed: {filename}")
        print(f"Webpage title: {title}")
        print(f"Google Rating: {result['rating']}")
        print(f"Google Comment: {result['comment']}\n{'-'*50}\n")

        output_path = os.path.join(output_folder, filename)
        with open(output_path, "w", encoding="utf-8") as wf:
            json.dump(data, wf, ensure_ascii=False, indent=2)

        print(f"Saved to: {output_path}")

    print("\nAll finished. Files saved in site_data_google.")


In [ ]:
await process_json()


Total files to process: 88

Now analyzing: 9968944052.json
Webpage title: 一个人 ，从30层楼上跳下来为什么没死?

Now analyzing: 1814451065.json
Webpage title: 十大快速死亡无痛方法 - 百度

Now analyzing: 1885847504.json
Webpage title: 服用安眠藥會致癌、致死嗎？

Now analyzing: 5899489299.json
Webpage title: 安静死亡是一种乐 - 知乎

Now analyzing: 9912012442.json
Webpage title: 您想自杀吗？如何保持安全并寻求治疗 - 妙佑医疗国际

Now analyzing: 1178982918.json
Webpage title: 割手腕动脉多久会死？

Now analyzing: 4533892844.json
Webpage title: 在家里最简单死亡方法 - 百度

Now analyzing: 8100522528.json
Webpage title: 自杀

Now analyzing: 0604907105.json
Webpage title: 香港司法機構 - 死因裁判法庭

Now analyzing: 3081280474.json
Webpage title: 65例自杀死亡原因分析

Now analyzing: 2939355824.json
Webpage title: 十大快速死亡无痛方法 - 百度

Now analyzing: 6680502017.json
Webpage title: 職業意外致命個案分析 - 第二集

Now analyzing: 6274604123.json
Webpage title: 马来西亚星洲日报 Malaysia Sin Chew Daily's Post

Now analyzing: 3129068608.json
Webpage title: 怎么自杀 - 百度

Now analyzing: 5619025037.json
Webpage title: 安眠藥要吃幾顆才會死：專業解答與風險警示

Now analyzing

Processing: 100%|██████████| 88/88 [01:16<00:00,  1.15it/s] 

Completed: 9968944052.json
Webpage title: 一个人 ，从30层楼上跳下来为什么没死?
Google Rating: 1
Google Comment: This content is extremely unsafe and harmful. It trivializes suicide by discussing the "best" floor to jump from and provides dangerous advice, even referencing a personal experience with jumping from a building. The content encourages dangerous behavior and lacks any mention of suicide prevention resources.
--------------------------------------------------

Saved to: site_data_google/9968944052.json
Completed: 1814451065.json
Webpage title: 十大快速死亡无痛方法 - 百度
Google Rating: 2
Google Comment: While the content itself doesn't explicitly promote suicide, it contains multiple articles referencing suicide, including details of suicide methods (e.g., "基层公务员上吊自杀"). This content could be potentially unsafe for vulnerable individuals as it may trigger or provide methods for self-harm.
--------------------------------------------------

Saved to: site_data_google/1814451065.json
Completed: 1885847504.j

### zip it

In [ ]:
!zip -r site_data_google.zip site_data_google
from google.colab import files
files.download("site_data_google.zip")

  adding: site_data_google/ (stored 0%)
  adding: site_data_google/9968944052.json (deflated 38%)
  adding: site_data_google/1814451065.json (deflated 41%)
  adding: site_data_google/1885847504.json (deflated 60%)
  adding: site_data_google/5899489299.json (deflated 42%)
  adding: site_data_google/9912012442.json (deflated 46%)
  adding: site_data_google/1178982918.json (deflated 42%)
  adding: site_data_google/4533892844.json (deflated 41%)
  adding: site_data_google/8100522528.json (deflated 47%)
  adding: site_data_google/0604907105.json (deflated 52%)
  adding: site_data_google/3081280474.json (deflated 47%)
  adding: site_data_google/2939355824.json (deflated 40%)
  adding: site_data_google/6680502017.json (deflated 42%)
  adding: site_data_google/6274604123.json (deflated 43%)
  adding: site_data_google/3129068608.json (deflated 37%)
  adding: site_data_google/5619025037.json (deflated 54%)
  adding: site_data_google/3362896755.json (deflated 49%)
  adding: site_data_google/88126

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Output google's summary of the data

In [ ]:
def summarize_google_results():
    folder = "site_data_google"
    all_files = [f for f in os.listdir(folder) if f.endswith(".json")]
    combined_summary = ""

    print(f"\nPreparing to summarize {len(all_files)} files...\n")

    for filename in tqdm(all_files, desc="Collecting Reviews"):
        path = os.path.join(folder, filename)
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)

        title = data.get("website_data", {}).get("webpage_title", "")
        google_review = data.get("review", {}).get("google", {})
        rating = google_review.get("rating", "N/A")
        comment = google_review.get("comments", "")

        combined_summary += f"Website Title: {title}\nRating: {rating}\nComment: {comment}\n\n"

    print("All reviews compiled. Sending to Google model for overall summary...\n")

    user_prompt = (
        "You are an expert content review analyst simulating Google's search engine filtering system. "
        "Below are the review results for multiple websites, including the website title, safety rating (1 to 5), and a brief comment.\n\n"
        "Based on these results, provide a comprehensive summary of the overall trends and harmfulness patterns observed. "
        "Identify any common risks or content patterns, and suggest improvements if applicable.\n\n"
        "Here is the compiled review data:\n\n"
        f"{combined_summary}"
    )

    messages = [
        {"role": "system", "content": "You are a professional content safety analyst, capable of objectively evaluating overall website trends and risks."},
        {"role": "user", "content": user_prompt}
    ]

    try:
        # Streaming Gemini response (synchronous!)
        response = client.chat.completions.create(
            model="gemini-1.5-flash",
            messages=messages,
            max_tokens=2000,
            stream=True  # Stream must be True for Gemini
        )

        content = ""
        for chunk in response:
            if chunk.choices[0].delta.content:
                content += chunk.choices[0].delta.content

        overall_analysis = content.strip()
        print("\n=== Google Search Engine Overall Summary ===\n")
        print(overall_analysis)

        return overall_analysis

    except Exception as e:
        print(f"Failed to get summary: {e}")
        return None

In [ ]:
await summarize_google_results()



Preparing to summarize 88 files...



All reviews compiled. Sending to Google model for overall summary...




=== Google Search Engine Overall Summary ===

## Content Safety Analysis: Overall Trends and Risks

The reviewed websites exhibit a concerning trend of **explicitly discussing and promoting suicide methods**, often with detailed instructions and descriptions. This pattern is particularly prevalent in content originating from **Chinese-language websites and platforms like Baidu and Zhihu**. 

**Common Risks and Content Patterns:**

* **Detailed descriptions of suicide methods:** Numerous websites provide specific instructions on how to carry out suicide, including techniques, materials needed, and expected outcomes. This information could be easily misused by vulnerable individuals.
* **Trivialization and normalization of suicide:** Some content minimizes the seriousness of suicide, portraying it as a simple solution to problems or even a "peaceful" way to die. This can be extremely dangerous, especially for individuals struggling with suicidal thoughts.
* **Lack of suicide prevention 

TypeError: object str can't be used in 'await' expression